<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/tda_2025_week2_ex4_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Exercise 2</h1>

Duplicate detection is one of the applications of embeddings. Let's try!

Grab the first 2000 examples of this dataset: https://huggingface.co/datasets/sentence-transformers/quora-duplicates (the "pairs" version) which contains examples of duplicate questions in Quora. Each example has an *anchor* and a *positive*.

Embed both anchors and positives with some embedding model. You can get away with a small, simple model like `all-MiniLM-L6-v2` which allows you to run this on CPU. Index the *positive* embeddings in FAISS (`IndexFlatL2` is quite enough!). Then, query the index with the *anchors* and evaluate how often the correct hit (i.e. the corresponding *positive*) is in top 1 and how often it is in top 5 (say). In other words, you are evaluating the accuracy of the retrieval.

In [ ]:
!pip3 install datasets
import datasets
ds=datasets.load_dataset("sentence-transformers/quora-duplicates","pair")

In [ ]:
import sentence_transformers
model=sentence_transformers.SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
examples=list(ds["train"])[:2000]
for ex in examples:
  print(ex["anchor"])
  print(ex["positive"])
  break

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


In [ ]:
anchors=model.encode([ex["anchor"] for ex in examples])
positives=model.encode([ex["positive"] for ex in examples])

In [ ]:
!pip3 install faiss-cpu
import faiss
index=faiss.IndexFlatL2(anchors.shape[1])
index.add(positives)

In [ ]:
D,N=index.search(anchors,k=5)

In [ ]:
#get some quick top1 accuracy
correct=0
correct_top1=0
for i in range(2000):
    if i in N[i]:
        correct+=1
    if i==N[i][0]:
        correct_top1+=1
print(f"Top1 correct: {correct_top1/2000}")
print(f"TopK correct: {correct/2000}")

Top1 correct: 0.797
TopK correct: 0.968
